# Model 3: Uplift Model

To use this model, please import the dataframe from the two first models.

In [ ]:
# Add a treatment column to the dataframe
# Randomly assign a 0 or 1 to each row. 1 for treatment, 0 for control
df['treatment'] = np.random.randint(2, size=len(df))


In [ ]:
df.treatment.value_counts()

In [ ]:
# Let's check the treatment's correlation to repeat purchases:
def correlation_treatment(df:pd.DataFrame):
    """Function to calculate the treatment's correlation
    """
    correlation = df[['treatment','REPEATER']].corr(method ='pearson') 
    return(pd.DataFrame(round(correlation.loc['REPEATER'] * 100,2)))

correlation_treatment(df)


In [ ]:
def declare_target_class(df:pd.DataFrame):
    """Function for declare the target class
    """
    # Control Non Repeaters (CN) : 0
    df['target_class'] = 0 
    # Control Repeaters (CR) : 1
    df.loc[(df.treatment == 0) & (df.REPEATER == 0),'target_class'] = 1 
    # Treatment Non Repeaters (TN) : 2
    df.loc[(df.treatment == 1) & (df.REPEATER == 1),'target_class'] = 2 
    # Treatment Repeaters (TR) : 3
    df.loc[(df.treatment == 1) & (df.REPEATER == 0),'target_class'] = 3 
    return df

df = declare_target_class(df)

In [ ]:
def split_data(df_model:pd.DataFrame):
    """Split data into training data and testing data
    """
    X = df_model.drop(['REPEATER', 'CLTV','target_class'],axis=1)
    y = df_model.REPEATER
    z = df_model.target_class
    X_train, X_test, \
    y_train, y_test, \
    z_train, z_test = train_test_split(X,
                                       y,
                                       z,
                                       test_size=0.3,
                                       random_state=42,
                                       stratify=df_model['treatment'])
    return X_train,X_test, y_train, y_test, z_train, z_test


def machine_learning(X_train:pd.DataFrame,
                     X_test:pd.DataFrame,
                     y_train:pd.DataFrame,
                     y_test:pd.DataFrame,
                     z_train:pd.DataFrame,
                     z_test:pd.DataFrame):
    """Machine learning process consists of 
    data training, and data testing process (i.e. prediction) with XGBoost (XGB) Algorithm
    """

    # prepare a new DataFrame
    prediction_results = pd.DataFrame(X_test).copy()
    
    
    # train the ETP model
    model_tp \
    = xgb.XGBClassifier().fit(X_train.drop('treatment', axis=1), y_train)  
    # prediction Process for ETP model 
    prediction_tp \
    = model_tp.predict(X_test.drop('treatment',axis=1))
    probability__tp \
    = model_tp.predict_proba(X_test.drop('treatment', axis=1))
    prediction_results['prediction_REPEATER'] = prediction_tp
    prediction_results['proba_REPEATER'] = probability__tp[:,1]
    
    
    # train the ETU model
    model_etu \
    = xgb.XGBClassifier().fit(X_train.drop('treatment', axis=1), z_train)
    # prediction Process for ETU model 
    prediction_etu \
    = model_etu.predict(X_test.drop('treatment', axis=1))
    probability__etu \
    = model_etu.predict_proba(X_test.drop('treatment', axis=1))
    prediction_results['prediction_target_class'] = prediction_etu
    prediction_results['proba_CN'] = probability__etu[:,0] 
    prediction_results['proba_CR'] = probability__etu[:,1] 
    prediction_results['proba_TN'] = probability__etu[:,2] 
    prediction_results['proba_TR'] = probability__etu[:,3]
    prediction_results['score_etu'] = prediction_results.eval('\
    proba_CN/(proba_CN+proba_CR) \
    + proba_TR/(proba_TN+proba_TR) \
    - proba_TN/(proba_TN+proba_TR) \
    - proba_CR/(proba_CN+proba_CR)')  
    # add the REPEATER and target class into dataframe as validation data
    prediction_results['REPEATER'] = y_test
    prediction_results['target_class'] = z_test
    return prediction_results


def predict(df_model:pd.DataFrame):
    """Combining data split and machine learning process with XGB
    """
    X_train, X_test, y_train, y_test, z_train, z_test = split_data(df_model)
    prediction_results = machine_learning(X_train,
                                          X_test,
                                          y_train,
                                          y_test,
                                          z_train,
                                          z_test)
    print("✔️Prediction succeeded")
    return prediction_results

# Machine Learning Modelling Process
print("Predicting dataset 1 ...")
prediction_results_1 = predict(df)

## Evaluating predictive performance

In [ ]:
def cm_evaluation(df:pd.DataFrame):
    """Confusion matrix evaluation
    """  
    print("===================================")
    print("1. ETP's confusion matrix result:")
    confusion_etp = confusion_matrix(df['REPEATER'], df['prediction_REPEATER'])
    df_confusion_etp = pd.DataFrame(confusion_etp, columns = ['Predicted True','Predicted False'], index = ['Actual True','Actual False'])
    print(df_confusion_etp)
    
    print("-----------------------------------")
    
    print("2. ETU's confusion matrix result:")   
    confusion_etu = multilabel_confusion_matrix(df['target_class'], df['prediction_target_class'])
    print("a. CN's confusion matrix:")  
    df_cn = pd.DataFrame(confusion_etu[0], columns = ['Predicted True','Predicted False'], index = ['Actual True','Actual False'])
    print(df_cn)
    print("b. CR's confusion matrix:") 
    df_cr = pd.DataFrame(confusion_etu[1], columns = ['Predicted True','Predicted False'], index = ['Actual True','Actual False'])
    print(df_cr) 
    print("c. TN's confusion matrix:")
    df_tn = pd.DataFrame(confusion_etu[2], columns = ['Predicted True','Predicted False'], index = ['Actual True','Actual False'])
    print(df_tn) 
    print("d. TR's confusion matrix:") 
    df_tr = pd.DataFrame(confusion_etu[3], columns = ['Predicted True','Predicted False'], index = ['Actual True','Actual False'])
    print(df_tr)
    
    print("===================================")

In [ ]:
# Confusion Matrix Evaluation
print("✔️Dataset 1")
cm_evaluation(prediction_results_1)

In [ ]:
def accuracy_evaluation(df:pd.DataFrame):
    """Accuracy evaluation
    """
    akurasi_cp = accuracy_score(df['REPEATER'],
                                df['prediction_REPEATER'])
    print('✔️ETP model accuracy: %.2f%%' % (akurasi_cp * 100.0))
    
    
    akurasi_uplift = accuracy_score(df['target_class'],
                                    df['prediction_target_class'])
    print('✔️ETU model accuracy: %.2f%%' % (akurasi_uplift * 100.0))

In [ ]:
# Accuracy Evaluation Process.
print("Dataset 1")
accuracy_evaluation(prediction_results_1)

## Evaluation of the prescriptive performance

In [ ]:
def sorting_data(df:pd.DataFrame):
    """Function to sort data
    """
    # Set up new DataFrames for ETP model and ETU model
    df_c = pd.DataFrame({'n':[], 'target_class':[]})
    df_u = df_c.copy()
    df_c['target_class'] = df['target_class']
    df_u['target_class'] = df['target_class']
    
    
    # Add quantiles
    df_c['n'] = df.proba_REPEATER.rank(pct=True, ascending=False)
    df_u['n'] = df.score_etu.rank(pct=True, ascending=False)
    df_c['score'] = df['proba_REPEATER']
    df_u['score'] = df['score_etu']
    
    
    # Ranking the data by deciles
    df_c = df_c.sort_values(by='n').reset_index(drop=True)
    df_u = df_u.sort_values(by='n').reset_index(drop=True)
    df_c['model'], df_u['model'] = 'CP', 'Uplift'
    return df_c, df_u


def calculating_qini(df:pd.DataFrame):
    """Function to measure the Qini value
    """
    # Calculate the C, T, CR, and TR
    C, T = sum(df['target_class'] <= 1), sum(df['target_class'] >= 2)
    df['cr'] = 0
    df['tr'] = 0
    df.loc[df.target_class  == 1,'cr'] = 1
    df.loc[df.target_class  == 3,'tr'] = 1
    df['cr/c'] = df.cr.cumsum() / C
    df['tr/t'] = df.tr.cumsum() / T
    

    # Calculate & add the qini value into the Dataframe
    df['uplift'] = df['tr/t'] - df['cr/c']
    df['random'] = df['n'] * df['uplift'].iloc[-1]
    qini_coef= df['uplift'].sum(skipna = True) - df['random'].sum(skipna = True)

    # Print the Qini coefficient
    print('✔️Qini coefficient = {} {}'.format(round(qini_coef, 2), '%'))
    
    # Add q0 into the Dataframe
    q0 = pd.DataFrame({'n':0, 'uplift':0, 'target_class': None}, index =[0])
    qini = pd.concat([q0, df]).reset_index(drop = True)
    return qini

def merging_data(df_c:pd.DataFrame, df_u:pd.DataFrame):
    """Function to add the 'Model' column and merge the dataframe into one
    """
    df_u['model'] = 'ETU'
    df_c['model'] = 'ETP'
    df = pd.concat([df_u, df_c]).sort_values(by='n').reset_index(drop = True)
    return df


def plot_qini(df:pd.DataFrame):
    """Function to plot the qini curve
    """
    print('\nPlotting the qini curve...')
    
    # Define the data that will be plotted
    order = ['ETU','ETP']
    ax = sns.lineplot(x='n', y=df.uplift, hue='model', data=df,
                      style='model', palette=['red','deepskyblue'],
                      style_order=order, hue_order = order)
    
    # Additional plot display settings
    handles, labels = ax.get_legend_handles_labels()
    plt.xlabel('Proportion targeted',fontsize=30)
    plt.ylabel('Uplift',fontsize=30)
    plt.subplots_adjust(right=1)
    plt.subplots_adjust(top=1)
    plt.legend(fontsize=30)
    ax.tick_params(labelsize=24)
    ax.legend(handles=handles[1:], labels=labels[1:])
    ax.plot([0,1], [0,df.loc[len(df) - 1,'uplift']],'--', color='grey')
    print('✔️Successfully plot the qini curve')
    return ax

def evaluation_qini(prediction_results:pd.DataFrame):
    """Function to combine all qini evaluation processes
    """
    df_c, df_u = sorting_data(prediction_results)
    print('ETP model (previous model):')
    qini_c = calculating_qini(df_c)
    print('\nETU model (our proposed model):')
    qini_u = calculating_qini(df_u)
    qini = merging_data(qini_c, qini_u)
    ax = plot_qini(qini)
    return ax, qini


In [ ]:
# Qini evaluation results for DataSet 1 with negative treatment correlation
ax, qini_1 = evaluation_qini(prediction_results_1)
plt.title('Qini Curve - Dataset 1',fontsize=20)

In [ ]:
# The process to inverse treatment's parameter
# Thus also inverse the treatment's correlation from negative to positive
df.treatment = df.treatment.replace({0: 1, 1: 0})

# Recalculate the treatment correlation
print("✔️Treatment correlation in dataset 1 (inverted):", correlation_treatment(df).iloc[0,0])

# Add the target class feature to all three datasets
df_inverse = declare_target_class(df)

# Do the prediction process once more time
prediction_results_inverse_1 = predict(df_inverse)

# Qini evaluation results for DataSet 1 with positive treatment correlation
ax, qini_inverse_1 = evaluation_qini(prediction_results_inverse_1)
plt.title('Qini Curve - Dataset 1',fontsize=20)

In [ ]:
# The process to inverse treatment's parameter
# Thus also inverse the treatment's correlation from negative to positive
df.treatment = df.treatment.replace({0: 1, 1: 0})

# Recalculate the treatment correlation
print("✔️Treatment correlation in dataset 1 (inverted):", correlation_treatment(df).iloc[0,0])

# Add the target class feature to all three datasets
df_inverse = declare_target_class(df)

# Do the prediction process once more time
prediction_results_inverse_1 = predict(df_inverse)

# Qini evaluation results for DataSet 1 with positive treatment correlation
ax, qini_inverse_1 = evaluation_qini(prediction_results_inverse_1)
plt.title('Qini Curve - Dataset 1',fontsize=20)